# UK Economy Agent-Based Model - Getting Started

This notebook demonstrates how to use the agent-based model (ABM) of the UK economy.

**Note**: This is a planning/example notebook. The ABM implementation is currently in progress.

## 1. Setup and Configuration

In [ ]:
# Import required libraries
import polars as pl
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

# (Future) ABM imports
# from companies_house_abm.abm.model import UKEconomyModel
# from companies_house_abm.abm.initialize import load_firms_from_parquet
# from companies_house_abm.abm.metrics import calculate_gdp, calculate_unemployment

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 2. Load Companies House Data

First, we load the financial data that will be used to initialize firm agents.

In [ ]:
# Load Companies House data
data_path = Path("../data/companies_house.parquet")

if data_path.exists():
    df = pl.read_parquet(data_path)
    print(f"Loaded {len(df):,} firm-period observations")
    print(f"Unique companies: {df['company_id'].n_unique():,}")
    print(f"\nData columns: {df.columns[:10]}...")
else:
    print("Data not found. Run the ingestion pipeline first:")
    print("  companies_house_abm ingest --output data/companies_house.parquet")

## 3. Explore Firm Data Distribution

Before running the ABM, let's understand the firm size distribution.

In [ ]:
# Get most recent data for each firm
if data_path.exists():
    latest = (
        df.sort("date")
        .group_by("company_id")
        .tail(1)
        .filter(pl.col("turnover_gross_operating_revenue").is_not_null())
        .filter(pl.col("turnover_gross_operating_revenue") > 0)
    )
    
    # Plot firm size distribution (log-log plot to see power law)
    turnover = latest["turnover_gross_operating_revenue"].to_numpy()
    turnover = turnover[turnover > 10000]  # Filter very small values
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Histogram
    ax1.hist(np.log10(turnover), bins=50, edgecolor='black', alpha=0.7)
    ax1.set_xlabel('Log10(Turnover)')
    ax1.set_ylabel('Frequency')
    ax1.set_title('Firm Size Distribution (Log Scale)')
    
    # Cumulative distribution (for Zipf's law)
    sorted_turnover = np.sort(turnover)[::-1]  # Descending order
    rank = np.arange(1, len(sorted_turnover) + 1)
    
    ax2.loglog(rank[::100], sorted_turnover[::100], 'o', markersize=2, alpha=0.5)
    ax2.set_xlabel('Rank (log scale)')
    ax2.set_ylabel('Turnover (log scale)')
    ax2.set_title('Zipf Plot: Does firm size follow power law?')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nFirm size statistics:")
    print(f"  Median turnover: £{np.median(turnover):,.0f}")
    print(f"  Mean turnover: £{np.mean(turnover):,.0f}")
    print(f"  90th percentile: £{np.percentile(turnover, 90):,.0f}")
    print(f"  99th percentile: £{np.percentile(turnover, 99):,.0f}")
    print(f"  Max turnover: £{np.max(turnover):,.0f}")

## 4. Initialize the ABM (Future)

Once the ABM is implemented, you'll be able to initialize it like this:

In [ ]:
# Example code (not yet implemented):

# # Load configuration
# config_path = "../config/model_parameters.yml"
# 
# # Initialize model
# model = UKEconomyModel(
#     config_path=config_path,
#     data_path=data_path,
#     seed=42
# )
# 
# print(f"Model initialized with:")
# print(f"  {len(model.firms)} firm agents")
# print(f"  {len(model.households)} household agents")
# print(f"  {len(model.banks)} bank agents")
# print(f"  Time horizon: {model.config['simulation']['periods']} periods")

## 5. Run a Simulation (Future)

Example of how to run the model:

In [ ]:
# Example simulation (not yet implemented):

# # Run for 100 quarters (25 years)
# results = []
# 
# for t in range(100):
#     # Step the model
#     model.step()
#     
#     # Collect statistics
#     stats = {
#         'period': t,
#         'gdp': model.calculate_gdp(),
#         'unemployment': model.calculate_unemployment(),
#         'inflation': model.calculate_inflation(),
#         'interest_rate': model.central_bank.policy_rate,
#         'bankruptcies': model.count_bankruptcies_this_period()
#     }
#     results.append(stats)
#     
#     if t % 10 == 0:
#         print(f"Period {t}: GDP={stats['gdp']:.2e}, "
#               f"Unemployment={stats['unemployment']:.1%}")
# 
# # Convert to DataFrame
# results_df = pl.DataFrame(results)

## 6. Visualize Results (Future)

In [ ]:
# Example visualization (not yet implemented):

# fig, axes = plt.subplots(2, 2, figsize=(14, 10))
# 
# # GDP growth
# ax = axes[0, 0]
# gdp = results_df['gdp'].to_numpy()
# gdp_growth = np.diff(gdp) / gdp[:-1] * 100
# ax.plot(gdp_growth)
# ax.set_ylabel('GDP Growth Rate (%)')
# ax.set_title('Economic Growth')
# ax.axhline(0, color='red', linestyle='--', alpha=0.3)
# 
# # Unemployment rate
# ax = axes[0, 1]
# ax.plot(results_df['unemployment'].to_numpy() * 100)
# ax.set_ylabel('Unemployment Rate (%)')
# ax.set_title('Labor Market')
# 
# # Inflation
# ax = axes[1, 0]
# ax.plot(results_df['inflation'].to_numpy() * 100)
# ax.set_ylabel('Inflation Rate (%)')
# ax.set_xlabel('Period')
# ax.set_title('Price Dynamics')
# ax.axhline(2, color='red', linestyle='--', alpha=0.3, label='Target')
# ax.legend()
# 
# # Interest rate
# ax = axes[1, 1]
# ax.plot(results_df['interest_rate'].to_numpy() * 100)
# ax.set_ylabel('Policy Rate (%)')
# ax.set_xlabel('Period')
# ax.set_title('Monetary Policy')
# 
# plt.tight_layout()
# plt.show()

## 7. Policy Experiments (Future)

Example of testing policy interventions:

In [ ]:
# Example policy experiment (not yet implemented):

# # Baseline scenario
# baseline = run_simulation(model, periods=200)
# 
# # Fiscal stimulus scenario
# model_stimulus = UKEconomyModel(config_path=config_path, data_path=data_path, seed=42)
# # Increase government spending by 5% at period 100
# model_stimulus.schedule_policy_change(
#     period=100,
#     policy='government_spending',
#     change=0.05  # +5%
# )
# stimulus = run_simulation(model_stimulus, periods=200)
# 
# # Compare outcomes
# fig, ax = plt.subplots(figsize=(12, 6))
# ax.plot(baseline['gdp'], label='Baseline', linewidth=2)
# ax.plot(stimulus['gdp'], label='Fiscal Stimulus (+5% G)', linewidth=2)
# ax.axvline(100, color='red', linestyle='--', alpha=0.3, label='Policy change')
# ax.set_xlabel('Period')
# ax.set_ylabel('GDP')
# ax.set_title('Fiscal Policy Experiment: Impact on GDP')
# ax.legend()
# plt.show()
# 
# # Calculate fiscal multiplier
# multiplier = (stimulus['gdp'][120] - baseline['gdp'][120]) / baseline['gdp'][100] / 0.05
# print(f"Estimated fiscal multiplier: {multiplier:.2f}")

## 8. Network Analysis (Future)

Analyze supply chain and credit networks:

In [ ]:
# Example network analysis (not yet implemented):

# import networkx as nx
# 
# # Extract supply chain network
# supply_chain = model.get_supply_chain_network()
# 
# # Calculate centrality measures
# in_degree_centrality = nx.in_degree_centrality(supply_chain)
# betweenness = nx.betweenness_centrality(supply_chain)
# 
# # Identify systemically important firms
# top_firms = sorted(betweenness.items(), key=lambda x: x[1], reverse=True)[:10]
# 
# print("Top 10 systemically important firms (by betweenness centrality):")
# for firm_id, centrality in top_firms:
#     firm = model.get_firm(firm_id)
#     print(f"  {firm.name}: {centrality:.4f} (Sector: {firm.sector})")
# 
# # Simulate supply chain shock
# print("\nSimulating supply shock to top firm...")
# model.apply_shock_to_firm(top_firms[0][0], shock_type='production', magnitude=-0.5)
# 
# # Run simulation to see propagation
# for t in range(20):
#     model.step()
# 
# print(f"GDP decline after shock: {(model.calculate_gdp() / baseline_gdp - 1)*100:.2f}%")

## Next Steps

To contribute to the ABM implementation:

1. Read the [ABM Design Document](../docs/abm-design.md)
2. Check the [Implementation Roadmap](../docs/implementation-roadmap.md)
3. Review the [Contributing Guide](../docs/contributing.md)
4. Pick a component to implement from the roadmap
5. Submit a pull request!

## References

- Farmer, J.D. & Foley, D. (2009). "The economy needs agent-based modelling." *Nature*
- Delli Gatti, D. et al. (2011). "Macroeconomics from the Bottom-up." Springer
- Poledna, S. et al. (2018). "The multi-layer network nature of systemic risk." *Nature Communications*